# Preprocessing pipeline

Even though Kaggle has worked to collect and ordered the data, the JSON files need some work before becoming trainable dataset

This will be organized in a three step process


P1 ==>
-  We will reduce the number of JSON files we will run on (1 in 100). This is to produce a more managable dataset as we are only testing the waters with our model. The full data can be reprocessed if the model is found to be good.
-  The info in CSVs will be brought in to become columns in panda dataset

P2 ==>
- Tokens will be created from the markdowns and codes, with slightly different process
- features will be added (length of text, num of lines of code vs comments, dummy variables to replace left values - which should be encoded differently when it's a right value)

P3 ==>
-  We will split the dataset into two. One for each type of cell. As it would be easier to build a model predicting just code and just MarkDown. We could then use their output to train our full model

P4 ==>
- replace order of cells with comparison relations `{ <, =, > }`, creating n(n-1)/2 combinations

P5 ==>
- Dataframes/ CSVs will be concatenated


Timing will be mandatory for long steps and recorded for purpose of optimizing re-running strategies




In [70]:
df1 = pd.read_json("data/train/00001756c60be8.json")

In [48]:
train_order = pd.read_csv("data/train_orders.csv")
train_order.cell_order=train_order.cell_order.str.split()
train_order.sample(5)

,id,cell_order
101544,ba8ec0edee076c,"[b60583e5, f410fa1b, 710020c5, 667d777e, b3f52..."
121325,df2df37e4c6910,"[5c08e7c6, 7d256d36, 338b13e6, d31ca04d, 09d9d..."
16508,1e18a15c453f3e,"[248c3be9, fb743e4c, 29a3b7c6, 76d33b56, 05423..."
98435,b4d7d0c828d1c1,"[fe995b22, 591cd7c0, df5f3d91, 0b4de330, 2b461..."
10369,1305ae8cd924b0,"[e0afd782, ff4b7e44, 75ae29d3, cd435475, a089f..."


## Step P1: Skim dataset and add order information

In [62]:
%%time
# 6 secs/ 2MB for 1 in 1000 skim rate
for index, row in train_order.iterrows():
    if index%1000==0:
        true_order = { h: i for i, h in enumerate(row["cell_order"])}

        # get dataframe
        df = pd.read_json(f"data/train/{row['id']}.json")
        df["true_order"] = df.index.map(true_order)
        #df["true_order_code"] = df[df.cell_type=="code"].rank()
        df.to_csv(f"data/train_P1/{row['id']}.csv")
        

CPU times: user 4.62 s, sys: 278 ms, total: 4.9 s
Wall time: 5.85 s


In [55]:
df["true_order"] = df.index.map(true_order)

In [59]:
df["true_order_by_type"] = df.groupby("cell_type")["true_order"].rank()

In [85]:
#df
preprocessing_unit(df1.iloc[-2]["source"])

NameError: name 'stopwords' is not defined

## Step P2: NLP preprocessing

In [66]:
import re
def preprocessing_unit(testtext: str):
    words = re.sub(r"[^a-zA-Z0-9]", " ", testtext.lower())
    words=nltk.word_tokenize(words)
    words=[w for w in words if w not in stopwords.words("english")]
    #print([w for w in words if w not in stopwords.words("english")])
    words = [words for words in words if not words.isdigit()]
    return words

In [67]:
preprocessing_unit()

AttributeError: 'int' object has no attribute 'lower'